<a href="https://colab.research.google.com/github/diegosfc91/Advanced-Machine-Learning/blob/main/Redes_Neuronales_Recurrentes_y_Prophet_para_Predicci%C3%B3n_de_Series_Temporales_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesamiento de datos:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Cargar los datasets
train_data = pd.read_csv('/content/train.csv', parse_dates=True, index_col=0)
test_data = pd.read_csv('/content/test.csv', parse_dates=True, index_col=0)

# Asegurarse de que el índice es de tipo datetime
if not pd.api.types.is_datetime64_any_dtype(train_data.index):
    train_data.index = pd.to_datetime(train_data.index)

if not pd.api.types.is_datetime64_any_dtype(test_data.index):
    test_data.index = pd.to_datetime(test_data.index)

# Exploración inicial
print("Información del conjunto de entrenamiento:")
print(train_data.info())
print("\nPrimeras filas del conjunto de entrenamiento:")
print(train_data.head())

print("\nInformación del conjunto de prueba:")
print(test_data.info())
print("\nPrimeras filas del conjunto de prueba:")
print(test_data.head())

# Verificar si las columnas contienen datos no numéricos
if not all(train_data.dtypes.apply(lambda x: pd.api.types.is_numeric_dtype(x))):
    print("\nAdvertencia: El conjunto de entrenamiento contiene columnas no numéricas.")
    print("Filtrando solo columnas numéricas...")
    train_data = train_data.select_dtypes(include=['number'])

if not all(test_data.dtypes.apply(lambda x: pd.api.types.is_numeric_dtype(x))):
    print("\nAdvertencia: El conjunto de prueba contiene columnas no numéricas.")
    print("Filtrando solo columnas numéricas...")
    test_data = test_data.select_dtypes(include=['number'])

# Visualización de la serie temporal
plt.figure(figsize=(14, 7))
plt.plot(train_data, label='Datos de entrenamiento')
plt.title('Serie temporal - Conjunto de entrenamiento')
plt.xlabel('Fecha')
plt.ylabel('Valor')
plt.legend()
plt.grid()
plt.show()

# Escalado de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.transform(test_data)

# Convertir de nuevo a DataFrames para facilitar la interpretación
scaled_train_df = pd.DataFrame(scaled_train_data, columns=train_data.columns, index=train_data.index)
scaled_test_df = pd.DataFrame(scaled_test_data, columns=test_data.columns, index=test_data.index)

# Visualización de los datos escalados
plt.figure(figsize=(14, 7))
plt.plot(scaled_train_df, label='Datos de entrenamiento escalados')
plt.title('Serie temporal escalada - Conjunto de entrenamiento')
plt.xlabel('Fecha')
plt.ylabel('Escala normalizada')
plt.legend()
plt.grid()
plt.show()

# Mostrar estadísticas descriptivas después del escalado
print("\nEstadísticas descriptivas de los datos escalados - Entrenamiento:")
print(scaled_train_df.describe())

print("\nEstadísticas descriptivas de los datos escalados - Prueba:")
print(scaled_test_df.describe())

# Preparación de datos para la LSTM
def create_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(data[i + sequence_length])
    return np.array(sequences), np.array(targets)

sequence_length = 10  # Número de pasos de tiempo

# Dividir datos en conjuntos de entrenamiento y prueba
train_values = scaled_train_df.values
X_train, y_train = create_sequences(train_values, sequence_length)

# Dividir el conjunto de prueba
test_values = scaled_test_df.values
X_test, y_test = create_sequences(test_values, sequence_length)

# Construcción del modelo LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, train_values.shape[1])),
    Dense(1)  # Predicción de un valor futuro
])

model.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Visualización de la pérdida durante el entrenamiento
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.grid()
plt.show()

# Evaluar el modelo en el conjunto de prueba
lstm_predictions = model.predict(X_test)

# Desescalar predicciones y valores reales
print("Dimensiones de las predicciones LSTM:", lstm_predictions.shape)
print("Dimensiones de los datos reales (test):", y_test.shape)

try:
    # Desescalar predicciones
    lstm_predictions_descaled = scaler.inverse_transform(
        np.hstack([lstm_predictions, np.zeros((lstm_predictions.shape[0], scaler.n_features_in_ - 1))])
    )[:, 0]  # Selecciona solo la primera columna

    # Desescalar valores reales (solo la primera columna)
    y_test_descaled = scaler.inverse_transform(
        np.hstack([y_test[:, 0].reshape(-1, 1), np.zeros((y_test.shape[0], scaler.n_features_in_ - 1))])
    )[:, 0]
except ValueError as e:
    print(f"Error en el desescalado: {e}")
    lstm_predictions_descaled, y_test_descaled = None, None

if lstm_predictions_descaled is not None and y_test_descaled is not None:
    # Calcular métricas para LSTM
    lstm_rmse = mean_squared_error(y_test_descaled, lstm_predictions_descaled, squared=False)
    lstm_mae = mean_absolute_error(y_test_descaled, lstm_predictions_descaled)
    print(f"LSTM RMSE: {lstm_rmse:.4f}")
    print(f"LSTM MAE: {lstm_mae:.4f}")
else:
    print("No se pueden calcular las métricas debido a un error en el desescalado.")

# Implementación de Prophet
prophet_train_data = train_data.iloc[:, 0].reset_index()  # Selecciona solo la primera columna
prophet_train_data.columns = ['ds', 'y']

# Crear y entrenar el modelo Prophet
prophet_model = Prophet()
prophet_model.fit(prophet_train_data)

# Crear un dataframe futuro para predicciones
future = prophet_model.make_future_dataframe(periods=len(test_data))
forecast = prophet_model.predict(future)

# Extraer predicciones para el conjunto de prueba
prophet_test_predictions = forecast[['ds', 'yhat']].tail(len(test_data))

# Calcular métricas para Prophet
prophet_rmse = mean_squared_error(test_data.values, prophet_test_predictions['yhat'].values, squared=False)
prophet_mae = mean_absolute_error(test_data.values, prophet_test_predictions['yhat'].values)
print(f"Prophet RMSE: {prophet_rmse:.4f}")
print(f"Prophet MAE: {prophet_mae:.4f}")

# Visualización de las predicciones
plt.figure(figsize=(14, 7))
plt.plot(test_data.index, test_data.values, label='Datos reales')
plt.plot(test_data.index, lstm_predictions_descaled, label='Predicciones LSTM')
plt.plot(prophet_test_predictions['ds'], prophet_test_predictions['yhat'], label='Predicciones Prophet')
plt.title('Comparación de predicciones: LSTM vs Prophet')
plt.xlabel('Fecha')
plt.ylabel('Valor')
plt.legend()
plt.grid()
plt.show()